In [17]:
import cv2
import numpy as np
from collections import deque
from ultralytics import YOLO
import torch
import warnings
from numpy.polynomial.polyutils import RankWarning  # <-- Added this import

# -------------------- YOLOv8 Model --------------------
model = YOLO("yolov8m.pt")
if torch.cuda.is_available():
    model.to("cuda")

vehicle_classes = [2, 3, 5, 7]  # car, motorcycle, bus, truck

# -------------------- Lane History Buffers --------------------
left_line_history = deque(maxlen=10)
right_line_history = deque(maxlen=10)

YM_PER_PIX = 30 / 720
XM_PER_PIX = 3.7 / 700

def preprocess_frame(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (13, 13), 0)
    edges = cv2.Canny(blur, 120, 200)
    return edges

def region_of_interest(edges):
    height, width = edges.shape
    mask = np.zeros_like(edges)
    polygon = np.array([[
        (int(0.1 * width), height),
        (int(0.45 * width), int(0.6 * height)),
        (int(0.55 * width), int(0.6 * height)),
        (int(0.9 * width), height)
    ]], dtype=np.int32)
    cv2.fillPoly(mask, polygon, 255)
    return cv2.bitwise_and(edges, mask)

def detect_lines(cropped_edges):
    return cv2.HoughLinesP(
        cropped_edges, 1, np.pi / 180, threshold=50,
        minLineLength=40, maxLineGap=150)

def average_slope_intercept(frame, lines):
    left_lines, right_lines = [], []
    if lines is None:
        return None, None
    height, width, _ = frame.shape
    for line in lines:
        for x1, y1, x2, y2 in line:
            slope = (y2 - y1) / (x2 - x1 + 1e-6)
            if abs(slope) < 0.5:
                continue
            if slope < 0:
                left_lines.append((x1, y1, x2, y2))
            else:
                right_lines.append((x1, y1, x2, y2))

    def make_line(points):
        if len(points) == 0:
            return None
        x_coords, y_coords = [], []
        for x1, y1, x2, y2 in points:
            x_coords += [x1, x2]
            y_coords += [y1, y2]
        poly = np.polyfit(y_coords, x_coords, 1)
        y1 = height
        y2 = int(height * 0.6)
        x1 = int(poly[0] * y1 + poly[1])
        x2 = int(poly[0] * y2 + poly[1])
        return np.array([x1, y1, x2, y2])

    return make_line(left_lines), make_line(right_lines)

def compute_curvature(line, y_eval):
    x = np.array([line[0], line[2]]) * XM_PER_PIX
    y = np.array([line[1], line[3]]) * YM_PER_PIX
    with warnings.catch_warnings():
        warnings.simplefilter('ignore', RankWarning)  # Use the imported RankWarning here
        fit = np.polyfit(y, x, 2)
    curvature = ((1 + (2 * fit[0] * y_eval * YM_PER_PIX + fit[1]) ** 2) ** 1.5) / np.abs(2 * fit[0])
    return curvature

def highlight_lane(frame, lines):
    overlay = np.zeros_like(frame)
    if len(lines) == 2:
        left = lines[0]
        right = lines[1]
        x1_left, y1_left, x2_left, y2_left = left
        x1_right, y1_right, x2_right, y2_right = right

        def line_params(x1, y1, x2, y2):
            A = (y2 - y1) / (x2 - x1 + 1e-6)
            B = y1 - A * x1
            return A, B

        A_left, B_left = line_params(x1_left, y1_left, x2_left, y2_left)
        A_right, B_right = line_params(x1_right, y1_right, x2_right, y2_right)

        try:
            x_intersect = int((B_right - B_left) / (A_left - A_right + 1e-6))
            y_intersect = int(A_left * x_intersect + B_left)
        except:
            x_intersect = int((x2_left + x2_right) / 2)
            y_intersect = int((y2_left + y2_right) / 2)

        pts = np.array([
            [x1_left, y1_left],
            [x_intersect, y_intersect],
            [x1_right, y1_right]
        ])
        cv2.fillPoly(overlay, [pts], (0, 255, 0))
        cv2.line(overlay, (x1_left, y1_left), (x_intersect, y_intersect), (0, 255, 0), 10)
        cv2.line(overlay, (x1_right, y1_right), (x_intersect, y_intersect), (0, 255, 0), 10)

    return cv2.addWeighted(frame, 1, overlay, 0.3, 0)

# -------------------- Video Setup --------------------
cap = cv2.VideoCapture("/home/amirparsa/Documents/Computer vision/Brisbane1.mp4")

screen_width = 1920
screen_height = 1080
margin = 20
edge_margin = 10

win_width = (screen_width - 2 * edge_margin - margin) // 2
win_height = (screen_height - 2 * edge_margin - margin) // 2

# Create and place windows
cv2.namedWindow("Original", cv2.WINDOW_NORMAL)
cv2.namedWindow("Edges", cv2.WINDOW_NORMAL)
cv2.namedWindow("Masked Edges", cv2.WINDOW_NORMAL)
cv2.namedWindow("Lane Highlight", cv2.WINDOW_NORMAL)

cv2.resizeWindow("Original", win_width, win_height)
cv2.resizeWindow("Edges", win_width, win_height)
cv2.resizeWindow("Masked Edges", win_width, win_height)
cv2.resizeWindow("Lane Highlight", win_width, win_height)

cv2.moveWindow("Original", edge_margin, edge_margin)
cv2.moveWindow("Edges", edge_margin + win_width + margin, edge_margin)
cv2.moveWindow("Lane Highlight", edge_margin, edge_margin + win_height + margin)       # swapped
cv2.moveWindow("Masked Edges", edge_margin + win_width + margin, edge_margin + win_height + margin)  # swapped


# -------------------- Main Loop --------------------
while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.resize(frame, (1280, 720))
    edges = preprocess_frame(frame)
    roi_edges = region_of_interest(edges)
    lines = detect_lines(roi_edges)
    left_line, right_line = average_slope_intercept(frame, lines)

    if left_line is not None:
        left_line_history.append(left_line)
    if right_line is not None:
        right_line_history.append(right_line)

    smoothed_lines = []
    if left_line_history:
        smoothed_lines.append(np.mean(left_line_history, axis=0).astype(int))
    if right_line_history:
        smoothed_lines.append(np.mean(right_line_history, axis=0).astype(int))

    lane_frame = highlight_lane(frame.copy(), smoothed_lines)

    # YOLO Detection
    results = model.predict(source=frame, conf=0.5, classes=vehicle_classes, verbose=False)
    for box in results[0].boxes:
        cls_id = int(box.cls)
        conf = float(box.conf)
        if conf > 0.5:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            label = model.names[cls_id]
            cv2.rectangle(lane_frame, (x1, y1), (x2, y2), (0, 255, 255), 2)
            cv2.putText(lane_frame, f"{label} {conf:.1%}", (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)

    # Curvature still calculated (but not shown)
    if len(smoothed_lines) == 2:
        y_eval = frame.shape[0]
        left_curv = compute_curvature(smoothed_lines[0], y_eval)
        right_curv = compute_curvature(smoothed_lines[1], y_eval)
        avg_curv = int((left_curv + right_curv) / 2)
        # Text display removed here

    # Resize for display
    img1 = cv2.resize(frame, (win_width, win_height))
    img2 = cv2.resize(cv2.cvtColor(edges, cv2.COLOR_GRAY2BGR), (win_width, win_height))
    img3 = cv2.resize(cv2.cvtColor(roi_edges, cv2.COLOR_GRAY2BGR), (win_width, win_height))
    img4 = cv2.resize(lane_frame, (win_width, win_height))

    # Show windows
    cv2.imshow("Original", img1)
    cv2.imshow("Edges", img2)
    cv2.imshow("Masked Edges", img3)
    cv2.imshow("Lane Highlight", img4)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
